In [1]:
import numpy as np
from scipy import sparse

In [2]:
beta = 0.8
tolerance = 10**-10
TOP_K = 5
dValueForExtraPolation = [8]

In [3]:
def getGraphData(file):
    graphEdges = dict()
    with open(file) as fp:
        line = fp.readline()
        while line:
            lineL = [ int(node) for node in line.strip().split('\t')]
            if graphEdges.get(lineL[0]):
                graphEdges[lineL[0]].append(lineL[1])
            else:
                graphEdges[lineL[0]] = [lineL[1]]
            line = fp.readline()
    return graphEdges


In [4]:
def createMatrixFromGraph(graph, size):
    matrix = np.zeros((size, size))
    for node in graph.keys():
        nodeKey = node
        nodeValues = graph.get(node)
        nodesCount = len(nodeValues)
        eachNodeVal = 1 / nodesCount
        # for handling duplicates
        valDict = dict()
        for val in nodeValues:
            if valDict.get(val):
                valDict[val] = valDict.get(val) + 1
            else:
                valDict[val] = 1

        for nodeVal in list(set(nodeValues)):
            # To match node and matrix indices: node = matrix indices + 1
            count = valDict.get(nodeVal)
            matrix[nodeVal-1][node-1] = count * float(eachNodeVal)
    return matrix

# matrix = createMatrixFromGraph(graphEdges, numberOfNodes)

In [5]:
def createMatrixFromGraphWithCount(graph, size):
    row = []
    col = []
    data = []
    for node in graph.keys():
        nodeKey = node
        nodeValues = graph.get(node)
        nodesCount = len(nodeValues)
        eachNodeVal = 1 / nodesCount
        # for handling duplicates
        valDict = dict()
        for val in nodeValues:
            if valDict.get(val):
                valDict[val] = valDict.get(val) + 1
            else:
                valDict[val] = 1
        for nodeVal in list(set(nodeValues)):
            # To match node and matrix indices: node = matrix indices + 1
            count = valDict.get(nodeVal)
            row.append(nodeVal-1)
            col.append(node-1)
            data.append(count)
    matrix = sparse.csr_matrix((data, (row, col)), shape=(size, size))
    return matrix

# matrixWithCount = createMatrixFromGraph(graphEdges, numberOfNodes)

In [6]:
def createMatrixFromGraphSparse(graph, size):
    row = []
    col = []
    data = []
    for node in graph.keys():
        nodeKey = node
        nodeValues = graph.get(node)
        nodesCount = len(nodeValues)
        eachNodeVal = 1 / nodesCount
        # for handling duplicates
        valDict = dict()
        for val in nodeValues:
            if valDict.get(val):
                valDict[val] = valDict.get(val) + 1
            else:
                valDict[val] = 1
        for nodeVal in list(set(nodeValues)):
            # To match node and matrix indices: node = matrix indices + 1
            count = valDict.get(nodeVal)
            row.append(nodeVal-1)
            col.append(node-1)
            data.append(count * float(eachNodeVal))
    matrix = sparse.csr_matrix((data, (row, col)), shape=(size, size))
    return matrix


In [7]:
def createMatrixFromGraphSparseWeighted(graph, size):
    row = []
    col = []
    data = []
    for node in graph.keys():
        nodeKey = node
        nodeValues = graph.get(node)
        nodesCount = len(nodeValues)
        eachNodeVal = 1 / nodesCount
        # for handling duplicates
        valDict = dict()
        for val in nodeValues:
            if valDict.get(val):
                valDict[val] = valDict.get(val) + 1
            else:
                valDict[val] = 1
        for nodeVal in list(set(nodeValues)):
            # To match node and matrix indices: node = matrix indices + 1
            count = valDict.get(nodeVal)
            row.append(nodeVal-1)
            col.append(node-1)
            data.append(count)
    matrix = sparse.csr_matrix((data, (row, col)), shape=(size, size))
    
    rowSum = matrix.sum(axis=1).T.tolist()[0]
    colSum = matrix.sum(axis=0).tolist()[0]
    row = []
    col = []
    data = []
    for node in graph.keys():
        nodeKey = node
        nodeValues = graph.get(node)
        nodesCount = len(nodeValues)
        eachNodeVal = 1 / nodesCount
        # for handling duplicates
        valDict = dict()
        for val in nodeValues:
            if valDict.get(val):
                valDict[val] = valDict.get(val) + 1
            else:
                valDict[val] = 1
        #nodeInWeightForNodeChildren
        sumNodeIn = 0
        for nodeVal in list(set(nodeValues)):
            sumNodeIn = sumNodeIn + rowSum[nodeVal-1]
        #nodeOutWeightForNodeChildren
        sumNodeOut = 0
        for nodeVal in list(set(nodeValues)):
            sumNodeOut = sumNodeIn + colSum[nodeVal-1]
        for nodeVal in list(set(nodeValues)):
            # To match node and matrix indices: node = matrix indices + 1
            nodeInC = rowSum[nodeVal-1]
            weightIn = float(nodeInC/sumNodeIn)
            
            nodeOutC = colSum[nodeVal-1]
            weightOut =float(nodeOutC/sumNodeOut)
            row.append(nodeVal-1)
            col.append(node-1)
            data.append(weightIn*weightOut)
    matrixNew = sparse.csr_matrix((data, (row, col)), shape=(size, size))
    return matrixNew



In [8]:
def powerIter(sparseMatrix):
    # Given, 1 is one vector with nx1 entries of value 1
    one = np.ones((numberOfNodes,1))

    # initialising the r0
    r = 1/ float(numberOfNodes) * one
    count = 0
    while True:
        count += 1
        rnew = ((1-beta)/numberOfNodes) * one + beta * sparseMatrix.dot(r)
        l1Norm = np.linalg.norm(abs(rnew - r), ord=1)
        if l1Norm < tolerance:
            break
        r = rnew
    print('Number of iterations it took is ', count)
    return r


In [9]:
def powerExtrapolation(transitionMatrix, numberOfNodes):
    dValues = dValueForExtraPolation
    dName = dValues[0]
    pageRankD = {}
    residualsD = {}
    for d in dValues:
        pageRankItr = []
        pageRankInitial = 1 / numberOfNodes * np.ones((numberOfNodes, 1))
        k = 0
        res = {}
        pageRankItr.append(pageRankInitial)
        while True:
            k += 1
            pageRankNext = (1 - beta) / numberOfNodes * np.ones((numberOfNodes, 1)) + beta * transitionMatrix.dot(pageRankInitial)
            if k == d + 2:
                pageRankNext = (pageRankNext - (beta**d)*pageRankItr[k - d]) / (1 - beta**d)
            error = np.linalg.norm(abs(pageRankNext - pageRankInitial), ord=1)
            res[k] = error
            if error < tolerance:
                pageRankItr.append(pageRankNext)
                break
            pageRankItr.append(pageRankNext)
            pageRankInitial = pageRankNext
        pageRankD[d] = pageRankItr
        residualsD[d] = res
        print('Number of iterations in power extrapolation it took is ', k)
    return pageRankD.get(dName)[-1]

In [10]:
def getValuesTuple(indices, r):
    return [(r[index][0], index+1) for index in indices]


In [11]:
def getTopBottomK(topK,r):
    # Top k page ranks with values
    topKPagesId = r.T[0].argsort()[-topK:][::-1]
    topKPages = getValuesTuple(topKPagesId,r)
    print('************Top k node ids with scores************')
    for pr in enumerate(topKPages):
        (index, (pRVal, nodeId)) = pr
        print('Page rank ', index+1, 'for page id ',nodeId+1,'with value is - ', pRVal)

    print('\n\n')

    # Bottom 5 page ranks with values
    bottomKPagesId = r.T[0].argsort()[:topK]
    bottomKPages = getValuesTuple(bottomKPagesId,r)
    print('************Bottom k node ids with scores************')
    for pr in enumerate(bottomKPages):
        (index, (pRVal, nodeId)) = pr
        print('Page rank ', index+1, 'for page id ',nodeId+1,'with value is - ', pRVal)



In [12]:
graphEdges = getGraphData('web-Stanford.txt')
numberOfNodes = sorted(graphEdges.keys(), reverse=True)[:1][0]
print("Number of nodes using is", numberOfNodes)
sparseMatrix = createMatrixFromGraphSparse(graphEdges, numberOfNodes)
sparseMatrixWeighted = createMatrixFromGraphSparseWeighted(graphEdges, numberOfNodes)
numberOfNonZeroEdges = sparse.csr_matrix.count_nonzero(sparseMatrix)
print('Number of non zero points is ',numberOfNonZeroEdges)
# powerRankVec, residualsR = powerExtrapolationPagerank(sparseMatrixWeighted, numberOfNodes)

Number of nodes using is 281903
Number of non zero points is  2312497


In [13]:
# Using normal page rank and power iteration
r = powerIter(sparseMatrix)
print("Normal page rank with power iteration")
getTopBottomK(TOP_K,r)


Number of iterations it took is  87
Normal page rank with power iteration
************Top k node ids with scores************
Page rank  1 for page id  89074 with value is -  0.010474629007599982
Page rank  2 for page id  226412 with value is -  0.009610237778224624
Page rank  3 for page id  241455 with value is -  0.008379659180575367
Page rank  4 for page id  134833 with value is -  0.0032578572592821694
Page rank  5 for page id  69359 with value is -  0.0027316738913733996



************Bottom k node ids with scores************
Page rank  1 for page id  2 with value is -  7.094638936087944e-07
Page rank  2 for page id  21387 with value is -  7.094638936087944e-07
Page rank  3 for page id  87192 with value is -  7.094638936087944e-07
Page rank  4 for page id  205726 with value is -  7.094638936087944e-07
Page rank  5 for page id  205720 with value is -  7.094638936087944e-07


In [14]:

# Using normal page rank and power extrapolation
r = powerExtrapolation(sparseMatrix, numberOfNodes)
print("Normal page rank with power extrapolation")
getTopBottomK(TOP_K,r)

Number of iterations in power extrapolation it took is  75
Normal page rank with power extrapolation
************Top k node ids with scores************
Page rank  1 for page id  89074 with value is -  0.010474629005360647
Page rank  2 for page id  226412 with value is -  0.009610237776892454
Page rank  3 for page id  241455 with value is -  0.00837965917897915
Page rank  4 for page id  134833 with value is -  0.003257857259166489
Page rank  5 for page id  69359 with value is -  0.002731673891306296



************Bottom k node ids with scores************
Page rank  1 for page id  2 with value is -  7.094638936087944e-07
Page rank  2 for page id  21387 with value is -  7.094638936087944e-07
Page rank  3 for page id  87192 with value is -  7.094638936087944e-07
Page rank  4 for page id  205726 with value is -  7.094638936087944e-07
Page rank  5 for page id  205720 with value is -  7.094638936087944e-07


In [15]:
# Using weighted page rank and power iteration
r = powerIter(sparseMatrixWeighted)
print("Normal page rank with power iteration")
getTopBottomK(TOP_K,r)



Number of iterations it took is  18
Normal page rank with power iteration
************Top k node ids with scores************
Page rank  1 for page id  82869 with value is -  0.00010306948712871268
Page rank  2 for page id  160278 with value is -  7.603265457011555e-05
Page rank  3 for page id  27685 with value is -  7.224273255526352e-05
Page rank  4 for page id  25636 with value is -  6.72899123907457e-05
Page rank  5 for page id  20707 with value is -  6.672859648514436e-05



************Bottom k node ids with scores************
Page rank  1 for page id  2 with value is -  7.094638936087944e-07
Page rank  2 for page id  185682 with value is -  7.094638936087944e-07
Page rank  3 for page id  185689 with value is -  7.094638936087944e-07
Page rank  4 for page id  185698 with value is -  7.094638936087944e-07
Page rank  5 for page id  185701 with value is -  7.094638936087944e-07


In [16]:

# Using weighted page rank and power extrapolation
r = powerExtrapolation(sparseMatrixWeighted,numberOfNodes)
print("Normal page rank with power extrapolation")
getTopBottomK(TOP_K,r)




Number of iterations in power extrapolation it took is  24
Normal page rank with power extrapolation
************Top k node ids with scores************
Page rank  1 for page id  82869 with value is -  0.00010306948712871268
Page rank  2 for page id  160278 with value is -  7.603265457011554e-05
Page rank  3 for page id  27685 with value is -  7.224273255526352e-05
Page rank  4 for page id  25636 with value is -  6.72899123907437e-05
Page rank  5 for page id  20707 with value is -  6.672859648514235e-05



************Bottom k node ids with scores************
Page rank  1 for page id  2 with value is -  7.094638936087944e-07
Page rank  2 for page id  185682 with value is -  7.094638936087944e-07
Page rank  3 for page id  185689 with value is -  7.094638936087944e-07
Page rank  4 for page id  185698 with value is -  7.094638936087944e-07
Page rank  5 for page id  185701 with value is -  7.094638936087944e-07
